In [ ]:
# 1. 安装 LLaMA-Factory 和相关依赖
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -e ".[torch,metrics]"
!pip install bitsandbytes

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 24605, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 24605 (delta 63), reused 22 (delta 22), pack-reused 24480 (from 2)
Receiving objects: 100% (24605/24605), 12.22 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (17694/17694), done.
/content/LLaMA-Factory
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.9 MB/s eta 0:00:00


In [ ]:
import os

# 1. 强制进入目录
%cd /content/LLaMA-Factory

# 2. 使用镜像加速下载
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 3. 禁用 W&B
os.environ["WANDB_DISABLED"] = "true"

# 4. 开始训练
!llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path Qwen/Qwen2.5-Coder-1.5B-Instruct \
    --dataset my_cleaner_data \
    --template qwen \
    --finetuning_type lora \
    --lora_target all \
    --output_dir saves/code_cleaner_v1 \
    --overwrite_output_dir \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --logging_steps 5 \
    --save_steps 100 \
    --learning_rate 1e-4 \
    --num_train_epochs 5.0 \
    --fp16

/content/LLaMA-Factory
2025-12-03 14:03:52.937764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764770632.958175   17378 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764770632.964256   17378 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764770632.980474   17378 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764770632.980509   17378 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764770632.980521   17378 computation_placer.cc:177]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. 设置路径
base_model_path = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
lora_path = "saves/code_cleaner_v1"

print("正在加载基座模型 (Qwen)...")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.float16
)

print("正在挂载你的 LoRA 权重...")
model = PeftModel.from_pretrained(model, lora_path)

# 2. 准备一个超级烂的代码案例
bad_code = """
def calc(x,y):
    # calc things
    a = x*y
    if a > 10: return a
    return 0
"""

# 3. 构造提示词
prompt = f"""请重构以下代码，使其符合 Google Style 规范并添加文档。
{bad_code}"""

messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# 4. 让模型生成
print("\n模型正在思考并重构代码...\n" + "="*50)
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    temperature=0.5
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# 5. 打印结果
print(response)
print("="*50)

⏳ 正在加载基座模型 (Qwen)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


🔗 正在挂载你的 LoRA 权重...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



🤖 模型正在思考并重构代码...
def calculate_product_and_check_limit(num1: float, num2: float) -> float:
    """Calculate the product of two numbers and check if it exceeds a limit.
    
    Args:
        num1: First number for multiplication.
        num2: Second number for multiplication.
        
    Returns:
        Product of num1 and num2 if it's less than or equal to 10; otherwise,
        returns 0.
        
    Examples:
        >>> calculate_product_and_check_limit(3, 4)
        12.0
        >>> calculate_product_and_check_limit(5, 6)
        0.0
    """
    product = num1 * num2
    
    if product <= 10:
        return product
    
    return 0.0


In [ ]:
import shutil
from google.colab import files

# 1. 压缩文件夹
print("正在打包模型权重...")
shutil.make_archive("my_code_cleaner_model", 'zip', "saves/code_cleaner_v1")

# 2. 触发下载
print("⬇开始下载到本地 (约 20MB)...")
files.download("my_code_cleaner_model.zip")

📦 正在打包模型权重...
⬇️ 开始下载到本地 (约 20MB)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
import shutil
import os

# 1. 挂载 Google Drive
drive.mount('/content/drive')

# 2. 定义源文件和目标路径
source_file = "my_code_cleaner_model.zip"
destination_path = "/content/drive/MyDrive/my_code_cleaner_model.zip"

# 3. 内网复制
print(f"在把 {source_file} 搬运到 Google Drive...")
shutil.copy(source_file, destination_path)

print(f"成功！请去你的 Google Drive 网盘根目录查看。")

Mounted at /content/drive
🚀 正在把 my_code_cleaner_model.zip 搬运到 Google Drive...
✅ 成功！请去你的 Google Drive 网盘根目录查看，文件已经在那里了。
